In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# Step 1. Import the WeatherPy_Database.csv file as a dataframe 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cidreira,BR,2022-08-01 00:04:57,-30.1811,-50.2056,58.93,88,88,9.89
1,1,Wanning,CN,2022-08-01 00:04:57,18.8003,110.3967,83.12,78,6,3.53
2,2,Dabat,ET,2022-08-01 00:04:58,12.9842,37.7650,54.43,87,95,3.89
3,3,Atuona,PF,2022-08-01 00:04:58,-9.8000,-139.0333,76.89,74,12,13.69
4,4,Zolotkovo,RU,2022-08-01 00:04:59,55.5281,41.1053,55.74,66,87,1.99


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Wanning,CN,2022-08-01 00:04:57,18.8003,110.3967,83.12,78,6,3.53
3,3,Atuona,PF,2022-08-01 00:04:58,-9.8000,-139.0333,76.89,74,12,13.69
6,6,Meulaboh,ID,2022-08-01 00:00:04,4.1363,96.1285,76.69,87,97,4.27
8,8,Butaritari,KI,2022-08-01 00:05:00,3.0707,172.7902,81.41,78,62,13.53
15,15,Makat,KZ,2022-08-01 00:05:04,47.6469,53.3463,80.78,36,90,7.87
...,...,...,...,...,...,...,...,...,...,...
690,690,Sri Aman,MY,2022-08-01 00:24:39,1.2376,111.4621,77.05,88,20,3.44
691,691,Farah,AF,2022-08-01 00:24:40,32.5000,62.5000,77.86,23,0,6.85
693,693,Phatthalung,TH,2022-08-01 00:25:41,7.5000,100.0833,78.66,85,100,4.18
694,694,Duluth,US,2022-08-01 00:21:12,46.7833,-92.1066,76.06,88,100,9.22


In [12]:
len(preferred_cities_df)

215

In [14]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Wanning,CN,2022-08-01 00:04:57,18.8003,110.3967,83.12,78,6,3.53
3,3,Atuona,PF,2022-08-01 00:04:58,-9.8000,-139.0333,76.89,74,12,13.69
6,6,Meulaboh,ID,2022-08-01 00:00:04,4.1363,96.1285,76.69,87,97,4.27
8,8,Butaritari,KI,2022-08-01 00:05:00,3.0707,172.7902,81.41,78,62,13.53
15,15,Makat,KZ,2022-08-01 00:05:04,47.6469,53.3463,80.78,36,90,7.87
...,...,...,...,...,...,...,...,...,...,...
690,690,Sri Aman,MY,2022-08-01 00:24:39,1.2376,111.4621,77.05,88,20,3.44
691,691,Farah,AF,2022-08-01 00:24:40,32.5000,62.5000,77.86,23,0,6.85
693,693,Phatthalung,TH,2022-08-01 00:25:41,7.5000,100.0833,78.66,85,100,4.18
694,694,Duluth,US,2022-08-01 00:21:12,46.7833,-92.1066,76.06,88,100,9.22


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
1,Wanning,CN,2022-08-01 00:04:57,18.8003,110.3967,83.12,78,6,3.53,
3,Atuona,PF,2022-08-01 00:04:58,-9.8000,-139.0333,76.89,74,12,13.69,
6,Meulaboh,ID,2022-08-01 00:00:04,4.1363,96.1285,76.69,87,97,4.27,
8,Butaritari,KI,2022-08-01 00:05:00,3.0707,172.7902,81.41,78,62,13.53,
15,Makat,KZ,2022-08-01 00:05:04,47.6469,53.3463,80.78,36,90,7.87,
24,Manzanillo,CU,2022-08-01 00:05:08,20.3433,-77.1167,80.62,78,89,9.66,
25,Vaini,TO,2022-08-01 00:05:09,-21.2000,-175.2000,76.23,86,95,11.83,
33,Hithadhoo,MV,2022-08-01 00:05:12,-0.6000,73.0833,80.78,74,100,16.67,
34,Kavaratti,IN,2022-08-01 00:00:24,10.5669,72.6420,80.01,81,100,23.69,
38,Banda Aceh,ID,2022-08-01 00:05:14,5.5577,95.3222,79.88,77,100,9.22,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")   

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
1,Wanning,CN,2022-08-01 00:04:57,18.8003,110.3967,83.12,78,6,3.53,Paradise Hotel
3,Atuona,PF,2022-08-01 00:04:58,-9.8000,-139.0333,76.89,74,12,13.69,Villa Enata
6,Meulaboh,ID,2022-08-01 00:00:04,4.1363,96.1285,76.69,87,97,4.27,Kost Pria
8,Butaritari,KI,2022-08-01 00:05:00,3.0707,172.7902,81.41,78,62,13.53,Isles Sunset Lodge
15,Makat,KZ,2022-08-01 00:05:04,47.6469,53.3463,80.78,36,90,7.87,Umit
...,...,...,...,...,...,...,...,...,...,...
686,Totness,SR,2022-08-01 00:24:37,5.8833,-56.3167,78.66,84,55,8.52,STAATS LOGEERGEBOUW TOTNESS
690,Sri Aman,MY,2022-08-01 00:24:39,1.2376,111.4621,77.05,88,20,3.44,Seri Simanggang Hotel
693,Phatthalung,TH,2022-08-01 00:25:41,7.5000,100.0833,78.66,85,100,4.18,Ozone Resort & Pool Villa
694,Duluth,US,2022-08-01 00:21:12,46.7833,-92.1066,76.06,88,100,9.22,South Pier Inn


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))